# Class IV - An introduction to GasModels.jl

*Los Alamos National Laboratory Grid Science Winter School, 2019*

Welcome! This tutorial will introduce you to the basics of the [GasModels.jl](https://github.com/lanl-ansi/GasModels.jl) package. If you haven't yet, work through [Class I - An introduction  to Julia](Class%20I%20-%20An%20introduction%20to%20Julia.ipynb) and [Class II - An introduction  to JuMP](Class%20II%20-%20An%20introduction%20to%20JuMP.ipynb) first.

As in Class II, run the following magic sauce to check we're good to go.

In [ ]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()
println("Excellent! Everything is good to go!")

### Background 

This [presentation](https://github.com/lanl-ansi/tutorial-grid-science-2019/blob/master/assets/infrastructure_optimization_in_julia.pdf) provides motivation and context for this notebook.

Some additional informaiton is also available at,
* [GasModels Documentation](https://lanl-ansi.github.io/GasModels.jl/stable/)


### Working with the Network Model

A 40-node gas network models is provided with this tutorial.  The MATLAB-like input data files can be viewed [here](../edit/data/gaslib40.m).

Similar to PowerModels, the `parse_file` function from GasModels is used to load a text files into the GasModels data model, 

In [ ]:
using GasModels

data = GasModels.parse_file("data/gaslib-40-E-ls.m")

In this case the file parser generated warning messages.  

The second warning block message indicates that the data file did not have any version information. Still the parser will load this data.

Now, lets look at some of the network data.  The data for the first pipe and compressor can be viewed as follows,

In [ ]:
data["pipe"]["1"]

In [ ]:
data["compressor"]["41"]

Here, connection can either be of type "pipe" in which case it has a length, diameter, and friction_factor associated with it or can be a compressor with a different set of fields as shown above.

The first junction in the network can be viewed as follows,

In [ ]:
data["junction"]["1"]

That's great, but looking at components one-by-one can get boring fast.

All InfrastructureModels packages provide a `print_summary` function that prints a table-like summary of the network data to the terminal.

In [ ]:
"prints the text summary for a data file or dictionary to stdout"
function print_summary(obj::Union{String, Dict{String,Any}}; kwargs...)
    GasModels.summary(stdout, obj; kwargs...)
end
print_summary(data)

### Solving the Gas Flow Problem (GF)

Before we can solve a gas flow problem, we need a solver.  The gas flow problem as defined in GasModels.jl is a Nonlinear Program (NLP). Hence, we require an NLP solver; in this case we will use [Ipopt](https://github.com/jump-dev/Ipopt.jl). 

In [ ]:
using JuMP
using Ipopt

ipopt_solver = optimizer_with_attributes(Ipopt.Optimizer, "tol"=>1e-6, "print_level"=>0)

result = run_gf(data, WPGasModel, ipopt_solver)

The result object contains a variety of useful information about the optimization problem solved, including the objective value and wall clock runtime.  Detailed documentation of the result dictionary is available [here](https://lanl-ansi.github.io/GasModels.jl/stable/result-data/), however most interesting point is the `solution` data.

In [ ]:
result["solution"]

Not every useful, lets try `print_summary`,

In [ ]:
GasModels.summary(stdout, result["solution"])

### Solving the Transient Optimal Gas Flow Problem (Transient OGF)

We now look at how we can solve the transient optimal control problem, which is formulated as an NLP. 

In [ ]:
using JuMP 
using Ipopt 

ipopt_solver = optimizer_with_attributes(Ipopt.Optimizer, "tol"=>1e-6)

mn_data = parse_files("data/case-6.m", "data/time-series-case-6b.csv", 
    spatial_discretization = 1e4)
    
result = run_transient_ogf(mn_data, WPGasModel, ipopt_solver)